In [ ]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.enable_eager_execution(config=config)
from tensorflow.contrib.data import shuffle_and_repeat, map_and_batch, prefetch_to_device, parallel_interleave
import os

In [ ]:
cores = os.cpu_count()
epochs=10
batch=128
shuffle_buff_size=5000
height, width = 100, 100

train_dir = "train/"
classes = os.listdir(train_dir)
classes = sorted(classes)
num_classes = len(classes)

In [ ]:
img_features = {
    'x': tf.FixedLenFeature([], dtype=tf.string, default_value=''),
    'y': tf.FixedLenFeature([], tf.int64)
}

In [ ]:
def parse_fn(example, height, width,num_classes, img_features):
    parsed = tf.parse_single_example(example, img_features)
    image, label = parsed["x"], parsed["y"]
    #print(label)
    image = tf.read_file(image)
    image = tf.image.decode_image(image)
    image = tf.image.resize_image_with_crop_or_pad(image,height,width)
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    #augmentation
    image = tf.image.random_flip_left_right(image)

    label = tf.one_hot(label, num_classes) 
    label = tf.cast(label, tf.float32)
    
    return image, label

In [ ]:
file_pattern = "train*.tfrecords"
files = tf.data.Dataset.list_files(file_pattern)
dataset = tf.data.TFRecordDataset(files, num_parallel_reads=cores)

dataset = dataset.apply(shuffle_and_repeat(shuffle_buff_size, epochs))
dataset = dataset.apply(
    map_and_batch(lambda x: parse_fn(x, height,width, num_classes, img_features), batch, num_parallel_batches=cores))
dataset = dataset.prefetch(4)

In [ ]:
layers = tf.keras.layers

model = tf.keras.Sequential()
model.add(layers.Conv2D(64,3, padding="same", activation="relu", input_shape=(height,width, 3)))
model.add(layers.Conv2D(64,3, padding="same", activation="relu"))
model.add(layers.MaxPool2D(2,2))

model.add(layers.Conv2D(128,3, padding="same", activation="relu"))
model.add(layers.Conv2D(128,3, padding="same", activation="relu"))
model.add(layers.MaxPool2D(2,2))

model.add(layers.Conv2D(256,3, padding="same", activation="relu"))
model.add(layers.Conv2D(256,3, padding="same", activation="relu"))
model.add(layers.Conv2D(256,3, padding="same", activation="relu"))
model.add(layers.MaxPool2D(2,2))

model.add(layers.Conv2D(512,3, padding="same", activation="relu"))
model.add(layers.Conv2D(512,3, padding="same", activation="relu"))
model.add(layers.Conv2D(512,3, padding="same", activation="relu"))
model.add(layers.MaxPool2D(2,2))

model.add(layers.Flatten())
model.add(layers.Dense(4096, activation="relu"))
model.add(layers.Dense(4096, activation="relu"))
model.add(layers.Dense(num_classes, activation="softmax"))

#model.summary()

In [ ]:
optimizer = tf.train.AdamOptimizer()
model.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy",
    metrics=['accuracy'])

In [ ]:
for (i, (x, y)) in enumerate(tf.contrib.eager.Iterator(dataset)):
    #print(i, x, y)
    loss, acc = model.train_on_batch(x,y)
    print(loss, acc)
    #print(y)

In [ ]:
loss, accuracy = model.evaluate(x_test,y_test)
print(loss, accuracy)